In [1]:
import sys,platform,os
import numpy as np, pandas as pd
import deepchem as dc
import rdkit
from rdkit import Chem

print("Python:",sys.version)
print("Platform:",platform.platform())
print("DeepChem:", dc.__version__)
print("RDKit:", rdkit.__version__)
print("NumPy:", np.__version__)
print("Pandas:", pd.__version__)

No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!
Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch_geometric'
Skipped loading modules with transformers dependency. No module named 'transformers'
cannot import name 'HuggingFaceModel' from 'deepchem.models.torch_models' (c:\Users\unine\.conda\envs\dc_win_gcn\lib\site-packages\deepchem\models\torch_models\__init__.py)
Skipped loading modules with pytorch-geometric dependency, missing a dependency. cannot import name 'DMPNN' from 'deepchem.models.torch_models' (c:\Users\unine\.conda\envs\dc_win_gcn\lib\site-packages\deepchem\models\torch_models\__init__.py)
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'lightning'
Skipped loading some Jax models, missing a dependency. No module named 'jax'
Skipped loading so

Python: 3.10.19 | packaged by conda-forge | (main, Oct 13 2025, 14:05:01) [MSC v.1944 64 bit (AMD64)]
Platform: Windows-10-10.0.26100-SP0
DeepChem: 2.8.0
RDKit: 2024.09.1
NumPy: 1.24.4
Pandas: 2.3.3


In [4]:
import shutil
import os
import warnings
# 壊れたキャッシュを削除
cache_dir = r"C:\Users\unine\AppData\Local\Temp\tox21-featurized"
if os.path.exists(cache_dir):
    print("古いキャッシュを削除中...")
    shutil.rmtree(cache_dir, ignore_errors=True)
    print("削除完了")

古いキャッシュを削除中...
削除完了


In [ ]:
import deepchem as dc
import numpy as np
from sklearn.metrics import roc_auc_score
import shutil 
import os
import warnings

# RDKitの警告を非表示
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')
warnings.filterwarnings('ignore')


# データ読み込み
print("\nデータ読み込み中...")
tox21_tasks, tox21_datasets, transformers = dc.molnet.load_tox21(
    featurizer='ECFP',
    splitter='random',
    reload=True
)

train_dataset, valid_dataset, test_dataset = tox21_datasets

print(f"訓練: {len(train_dataset)}, 検証: {len(valid_dataset)}, テスト: {len(test_dataset)}")
print(f"タスク数: {len(tox21_tasks)}")

# モデル構築
print("\nモデル構築中...")
model = dc.models.MultitaskClassifier(
    n_tasks=len(tox21_tasks),
    n_features=1024,
    layer_sizes=[1000],
    dropouts=0.25,
    learning_rate=0.001
)

# 学習
print("\n学習中...")
metric = dc.metrics.Metric(dc.metrics.roc_auc_score, np.mean)

for epoch in range(1, 51):
    model.fit(train_dataset, nb_epoch=1)
    if epoch % 10 == 0:
        valid_scores = model.evaluate(valid_dataset, [metric])
        print(f"Epoch {epoch}: Valid AUC = {valid_scores['mean-roc_auc_score']:.4f}")

# 評価
print("\n最終評価:")
test_scores = model.evaluate(test_dataset, [metric])
print(f"Test AUC-ROC: {test_scores['mean-roc_auc_score']:.4f}")

# タスクごとの結果
print("\nタスクごとの結果:")
y_pred = model.predict(test_dataset)

# 予測が2次元（クラス確率）の場合、陽性クラスの確率を取得
if len(y_pred.shape) == 3:
    y_pred = y_pred[:, :, 1]

for i, task in enumerate(tox21_tasks):
    valid_idx = test_dataset.w[:, i] != 0
    if valid_idx.sum() > 0:
        y_true = test_dataset.y[valid_idx, i]
        y_score = y_pred[valid_idx, i]
        try:
            auc = roc_auc_score(y_true, y_score)
            print(f"{task}: {auc:.4f}")
        except:
            print(f"{task}: 計算不可")

# モデル保存
model.save_checkpoint(model_dir='tox21_model')
print("\nモデル保存完了")


データ読み込み中...


Failed to featurize datapoint 1322, None. Appending empty array
Exception message: Python argument types in
    rdkit.Chem.rdmolfiles.CanonicalRankAtoms(NoneType)
did not match C++ signature:
    CanonicalRankAtoms(class RDKit::ROMol mol, bool breakTies=True, bool includeChirality=True, bool includeIsotopes=True, bool includeAtomMaps=True, bool includeChiralPresence=False)
Failed to featurize datapoint 2290, None. Appending empty array
Exception message: Python argument types in
    rdkit.Chem.rdmolfiles.CanonicalRankAtoms(NoneType)
did not match C++ signature:
    CanonicalRankAtoms(class RDKit::ROMol mol, bool breakTies=True, bool includeChirality=True, bool includeIsotopes=True, bool includeAtomMaps=True, bool includeChiralPresence=False)
Failed to featurize datapoint 2297, None. Appending empty array
Exception message: Python argument types in
    rdkit.Chem.rdmolfiles.CanonicalRankAtoms(NoneType)
did not match C++ signature:
    CanonicalRankAtoms(class RDKit::ROMol mol, bool brea

訓練: 6258, 検証: 782, テスト: 783
タスク数: 12

モデル構築中...

学習中...
Epoch 10: Valid AUC = 0.7792
Epoch 20: Valid AUC = 0.7713
Epoch 30: Valid AUC = 0.7681
Epoch 40: Valid AUC = 0.7633
Epoch 50: Valid AUC = 0.7595

最終評価:
Test AUC-ROC: 0.7708

タスクごとの結果:
NR-AR: 0.7452
NR-AR-LBD: 0.8175
NR-AhR: 0.8426
NR-Aromatase: 0.7844
NR-ER: 0.6931
NR-ER-LBD: 0.8529
NR-PPAR-gamma: 0.7216
SR-ARE: 0.7579
SR-ATAD5: 0.8513
SR-HSE: 0.7075
SR-MMP: 0.8302
SR-p53: 0.7909

モデル保存完了
